In [ ]:
import numpy as np
from scipy.stats import sampling
from scipy import stats
from distributions import StdNorm, Gamma, Beta, GenNorm

In [ ]:
# Set the uniform random number generator
urng1 = np.random.Generator(np.random.MT19937(0))
urng2 = np.random.Generator(np.random.PCG64(0))

In [ ]:
dists = [StdNorm, Gamma, Beta]
stdnorm_params = [()]
gamma_params = [(0.05,), (0.5,), (3.0,)]
beta_params = [(0.5, 0.5), (0.5, 1.0), (1.3, 1.2), (3.0, 2.0)]
dist_params = [stdnorm_params, gamma_params, beta_params]

In [ ]:
methods = [sampling.NumericalInversePolynomial,
           sampling.NumericalInverseHermite,
           sampling.TransformedDensityRejection,
           sampling.SimpleRatioUniforms]
method_names = ["PINV", "HINV", "TDR", "SROU"]

In [ ]:
sample_size = 1_000_000  # sample 1mn variates

In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for urng in [urng1, urng2]:
        print(f"Using URNG {urng.bit_generator.__class__.__name__}")
        for Dist, params in zip(dists, dist_params):
            for i, param in enumerate(params):
                dist = Dist(*param)
                for Method, method_name in zip(methods, method_names):
                    print(f"{method_name}, {dist}{param} [setup]    : ", end="")
                    try:
                        if method_name == "HINV":
                            method = Method(dist, u_resolution=1e-10, random_state=urng)
                            %timeit Method(dist, u_resolution=1e-10, random_state=urng)
                        elif method_name == "SROU":
                            mode = dist.mode()
                            method = Method(dist, mode=mode, random_state=urng)
                            %timeit Method(dist, mode=mode, random_state=urng)
                        else:
                            method = Method(dist, random_state=urng)
                            %timeit Method(dist, random_state=urng)
                        print(f"{method_name}, {dist}{param} [sampling] : ", end="")
                        %timeit method.rvs(sample_size)
                    except sampling.UNURANError:
                        print("Failed")

In [ ]:
for urng in [urng1, urng2]:
    print(f"Using URNG {urng.bit_generator.__class__.__name__}")
    for params in stdnorm_params:
        %timeit urng.standard_normal(*params, size=sample_size)

In [ ]:
for urng in [urng1, urng2]:
    print(f"Using URNG {urng.bit_generator.__class__.__name__}")
    for params in gamma_params:
        print(f"Gamma{param}    : ", end="")
        %timeit urng.gamma(*params, size=sample_size)

In [ ]:
for urng in [urng1, urng2]:
    print(f"Using URNG {urng.bit_generator.__class__.__name__}")
    for params in beta_params:
        print(f"Beta{param}    : ", end="")
        %timeit urng.beta(*params, size=sample_size)

In [ ]:
gennorm_params = [0.25, 0.45, 0.75, 1., 1.5, 2, 5, 8]

In [ ]:
def np_gennorm(beta, size, random_state):
    z = random_state.gamma(1/beta, size=size)
    y = z ** (1/beta)
    # convert y to array to ensure masking support
    y = np.asarray(y)
    mask = random_state.random(size=y.shape) < 0.5
    y[mask] = -y[mask]
    return y

In [ ]:
urng = np.random.Generator(np.random.PCG64(0))

In [ ]:
for param in gennorm_params:
    print(f"[NP]    gennorm({param})    : ", end="")
    %timeit np_gennorm(param, size=1_000_000, random_state=urng)
    print(f"[SciPy] gennorm({param})    : ", end="")
    dist = stats.gennorm(param)
    %timeit dist.rvs(size=1_000_000, random_state=urng)
    print(f"[PINV]  gennorm({param})    : ", end="")
    dist = GenNorm(param)
    %timeit rng = sampling.NumericalInversePolynomial(dist, random_state=urng); rng.rvs(1_000_000)